# Práctica 4: Clasificación semi-supervisada

En esta práctica exploraremos **clasificación semi-supervisada**, un enfoque que combina datos **etiquetados** y datos **no etiquetados** para mejorar el rendimiento de nuestros modelos.

## Objetivos
1. **Tarea 1**: Implementar un método de clasificación semi-supervisada (por ejemplo, *self-training*).
2. **Tarea 2**: Comparar al menos dos métodos disponibles en librerías (por ejemplo, *Label Propagation* y *Label Spreading*) en tres datasets.

Empezaremos con un breve repaso de conceptos, y luego desarrollaremos las secciones solicitadas.

## 1. Breve repaso teórico
La **clasificación semi-supervisada** busca aprovechar la gran cantidad de datos no etiquetados para mejorar la capacidad de generalización de un modelo. La idea principal es que, si contamos con pocas etiquetas pero tenemos muchos ejemplos sin etiquetar, podemos iterativamente asignar pseudo-etiquetas (cuando estemos razonablemente seguros) y así expandir nuestro conjunto etiquetado.

### Principales enfoques
- **Self-training**: Entrenamos un clasificador con los datos etiquetados y predecimos sobre los no etiquetados. Agregamos al conjunto de entrenamiento aquellos ejemplos cuya predicción sea más confiable.
- **Label Propagation** y **Label Spreading** (métodos *graph-based*): Construyen un grafo de similitud entre instancias y propagan las etiquetas de los ejemplos etiquetados a sus vecinos más cercanos.
- **Co-training**: Entrena dos (o más) clasificadores con diferentes subconjuntos de características (o vistas) y cada uno va etiquetando para el otro.
- **S3VM**: Extensión semi-supervisada de las *Support Vector Machines*, donde los datos no etiquetados contribuyen a maximizar el margen en las regiones de baja densidad.


# TAREA 1: Implementación de un método semi-supervisado

En esta sección implementamos un método de **Self-Training** en Python, utilizando un clasificador base de scikit-learn (por ejemplo, un **RandomForestClassifier**).

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier

def self_training(X_labeled, y_labeled, X_unlabeled, base_classifier, 
                  confidence_threshold=0.9, max_iter=10):
    """
    Implementación simple de Self-Training.
    --------------------------------------------------
    X_labeled, y_labeled: Datos (X) y etiquetas (y) para la parte etiquetada.
    X_unlabeled: Datos sin etiqueta.
    base_classifier: Clasificador supervisado que soporte al menos predict_proba (o predict).
    confidence_threshold: Umbral mínimo de probabilidad para aceptar pseudoejemplos.
    max_iter: Número máximo de iteraciones.
    """
    X_l = X_labeled.copy()
    y_l = y_labeled.copy()
    X_u = X_unlabeled.copy()

    for iteration in range(max_iter):
        # Entrenamos el clasificador con los datos etiquetados
        base_classifier.fit(X_l, y_l)

        # Predecimos probabilidades sobre el conjunto no etiquetado
        if hasattr(base_classifier, "predict_proba"):
            probs = base_classifier.predict_proba(X_u)
            pred_labels = np.argmax(probs, axis=1)
            max_probs = np.max(probs, axis=1)
        else:
            # Si no tiene predict_proba, usamos predict y asumimos confianza=1
            pred_labels = base_classifier.predict(X_u)
            max_probs = np.ones(len(X_u))

        # Seleccionamos las instancias donde la confianza >= confidence_threshold
        high_conf_idx = np.where(max_probs >= confidence_threshold)[0]
        if len(high_conf_idx) == 0:
            # Si no hay instancias con suficiente confianza, detenemos
            print(f"Iteración {iteration}: no se encontraron instancias con confianza >= {confidence_threshold}.")
            break

        # Agregamos esas instancias al conjunto etiquetado
        X_l = np.vstack([X_l, X_u[high_conf_idx]])
        y_l = np.hstack([y_l, pred_labels[high_conf_idx]])

        # Quitamos dichas instancias de la lista de no etiquetados
        X_u = np.delete(X_u, high_conf_idx, axis=0)

        print(f"Iteración {iteration}: se agregaron {len(high_conf_idx)} instancias con alta confianza.")

    # Al finalizar, retornamos el clasificador entrenado
    return base_classifier


### Ejemplo de uso con datos sintéticos

En este ejemplo:
- Generamos un dataset sintético con `make_classification`.
- Suponemos que solo una parte de las instancias están etiquetadas.
- Aplicamos *self-training* para agregar pseudo-etiquetas a los datos no etiquetados.
- Finalmente, medimos el desempeño en un conjunto de prueba.

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Generar dataset sintético
X, y = make_classification(n_samples=1000, n_features=10, n_informative=3,
                           n_classes=2, random_state=42)

# Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Tomar solo el 20% de y_train como etiquetado
labeled_percentage = 0.2
num_labeled = int(len(y_train) * labeled_percentage)
X_labeled = X_train[:num_labeled]
y_labeled = y_train[:num_labeled]
X_unlabeled = X_train[num_labeled:]  # no se usan etiquetas para este subconjunto

# Entrenar con self-training
rf = RandomForestClassifier(n_estimators=100, random_state=42)
self_trained_model = self_training(X_labeled, y_labeled, X_unlabeled, rf,
                                   confidence_threshold=0.9,
                                   max_iter=10)

# Evaluar en el conjunto de prueba
y_pred = self_trained_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Exactitud en test después de Self-Training:", accuracy)

# TAREA 2: Comparación de métodos
Para esta tarea:
1. Seleccionamos **dos algoritmos** semi-supervisados (por ejemplo, *LabelPropagation* y *LabelSpreading* de scikit-learn).
2. Aplicamos cada método a **varios datasets** (idealmente 3 o más) y comparamos resultados.

Aquí mostraremos un **ejemplo** con un solo dataset de `scikit-learn` (digits), pero la idea es repetir con más conjuntos de datos (por ejemplo, *Wine*, *Breast Cancer*, etc.) y luego **comparar**.


In [ ]:
from sklearn.datasets import load_digits
from sklearn.semi_supervised import LabelPropagation, LabelSpreading

# Cargar dataset (digits)
digits = load_digits()
X_dig = digits.data
y_dig = digits.target

# Dividir en train/test
X_train_dig, X_test_dig, y_train_dig, y_test_dig = train_test_split(
    X_dig, y_dig, test_size=0.3, random_state=42)

# Simular que solo un 10% de la parte de entrenamiento está etiquetada
num_label_dig = int(0.1 * len(y_train_dig))
labels_dig = np.copy(y_train_dig)
labels_dig[num_label_dig:] = -1  # usar -1 para indicar sin etiqueta

print(f"Número total de instancias de entrenamiento: {len(y_train_dig)}")
print(f"Instancias etiquetadas: {num_label_dig}")
print(f"Instancias no etiquetadas: {len(y_train_dig) - num_label_dig}")

# 1. Label Propagation
lp_model = LabelPropagation(kernel='rbf', gamma=20, max_iter=1000)
lp_model.fit(X_train_dig, labels_dig)
y_pred_lp = lp_model.predict(X_test_dig)
acc_lp = accuracy_score(y_test_dig, y_pred_lp)

# 2. Label Spreading
ls_model = LabelSpreading(kernel='rbf', alpha=0.2, max_iter=1000)
ls_model.fit(X_train_dig, labels_dig)
y_pred_ls = ls_model.predict(X_test_dig)
acc_ls = accuracy_score(y_test_dig, y_pred_ls)

print("\nResultados con Label Propagation:")
print("Accuracy:", acc_lp)

print("\nResultados con Label Spreading:")
print("Accuracy:", acc_ls)

## Discusión de resultados
En esta sección se espera:
- **Analizar** la precisión (o cualquier otra métrica como F1-score, etc.).
- **Comparar** cada método (Label Propagation, Label Spreading) y argumentar por qué uno puede superar a otro.
- **Repetir** el procedimiento con otros dos datasets (por ejemplo, [*Wine*](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html), [*Breast Cancer*](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html)) y mostrar tablas/gráficas de comparación.

Al final, se podrían extraer **conclusiones** como:
- El método *Label Propagation* puede ser más rápido, pero a veces más sensible al número de instancias mal etiquetadas.
- *Label Spreading* tiende a suavizar la propagación de las etiquetas, lo cual puede ayudar a reducir el ruido.
- Si la estructura de los datos forma *clusters* bien diferenciados, estos métodos *graph-based* aprovechan la asunción de que ejemplos cercanos comparten etiqueta.

En **prácticas reales**, se pueden probar distintos valores de parámetros (ej. `gamma`, `alpha`, número de vecinos, etc.) y diferentes porcentajes de datos etiquetados (5%, 10%, 20%, etc.) para observar el impacto en el rendimiento.

## Conclusiones
En esta práctica se han mostrado:
1. **Self-Training** como ejemplo de método de clasificación semi-supervisada implementado desde cero.
2. Comparaciones de métodos disponibles en **scikit-learn**: *Label Propagation* y *Label Spreading*.

### Puntos clave
- La **clasificación semi-supervisada** puede brindar mejoras sustanciales cuando se dispone de pocos datos etiquetados y abundantes datos no etiquetados.
- Es fundamental **validar** que nuestros supuestos (por ejemplo, "instancias cercanas tienen la misma etiqueta") se cumplan de forma razonable.
- Añadir datos no etiquetados puede **empeorar** el rendimiento si el modelo realiza asunciones incorrectas.

¡Y con esto concluimos la práctica de clasificación semi-supervisada!